In [0]:
# Import all the required modules for this network
from __future__ import print_function         
import torch                                  # PyTorch library
import torch.nn as nn                         # PyTorch Neural Network library
import torch.nn.functional as F               # Different funtions within neural networks such as Activation and Loss functions 
import torch.optim as optim                   # Diffrerent Optimizer algorithms such as Adam, RMSProp
from torchvision import datasets, transforms  # Datasets for storing data and transforms for transforming the data such as scale, resize etc

In [0]:
class Net(nn.Module):
    ''' Custom Neural Network 
    This is a class extended from pytorch inbuilt class 'Module'. '''
    def __init__(self):
        # Call to base class constrcutor
        super(Net, self).__init__()

        # Defining different layers such as convolution and pool
        # Conv2d represents a Convolution on an image of 2D.
        #     First Parameter: Input no of channels           - [1] - Black/White image
        #     Second Parameter: Output no of channels         - [32]
        #     Third Parameter: Kernel size such as 3 for 3x3  - [3]
        # MaxPool represents Max pooling which reduces the image size.
        # Last Conv2D: It has output channel size of 10 which is the no of classes, here it is digits 0-9.
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)         # input - 28x28x1,  Output - 26x26x32,   RF - 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)        # input - 26x26x32, Output - 26x26x64,   RF - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                     # input - 26x26x64, Output - 14x14x64,   RF - 10x10

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)       # input - 14x14x64,  Output - 12x12x128, RF - 12x12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)      # input - 12x12x128, Output - 10x10x256, RF - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                     # input - 10x10x256, Output - 5x5x256,   RF - 28x28

        self.conv5 = nn.Conv2d(256, 512, 3)                 # input - 5x5x256, Output - 3x3x512,     RF - 30x30
        self.conv6 = nn.Conv2d(512, 1024, 3)                # input - 3x3x512, Output - 1x1x1024,    RF - 32x32

        self.conv7 = nn.Conv2d(1024, 10, 3)                 # input - 1x1x1024, Output - 1x1x10,     RF - 34x34

    
    def forward(self, x):
        ''' Performs forward propagation in neural netowrk
            An overwridden of base class forward function'''
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # Block 1: Conv -> Relu -> Conv -> Relu -> Pool
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # Block 2: Conv -> Relu -> Conv -> Relu -> Pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             # Block 3: Conv -> Relu -> Conv -> Relu
        x = F.relu(self.conv7(x))                                 # Block 4:Conv -> Relu
        x = x.view(-1, 10)                                        # View reshapes a tensor, here we are specifying 10 for num of classes

        return F.log_softmax(x) # Passes a tensor to Softmax function to convert activations to probabilistic scores

In [4]:
# Install torchsummary, a package for printing model summary similar to keras model.summary
!pip install torchsummary
from torchsummary import summary

# Check if 'cuda' is present or not
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Create a neural network and assign it to device memory
model = Net().to(device)

# print model summary and pass model input size which is the image size
# In this case, (1, 28, 28) --> (channel, height, width)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:

# A seed it assigned for reproduciblity. For e.g same value is returned for every call to torch.rand(2)
torch.manual_seed(1)
batch_size = 128  # Batch size is the no of samples to be loaded within every forward/backward pass

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}         # num_workers allows parallel processing of items within a batch. pin_memory=True is used for faster copies from a host 
train_loader = torch.utils.data.DataLoader(                                 # Loading the train data. DataLoader is used to load the data which can be iterated.
    datasets.MNIST('../data', train=True, download=True,                    # Here we are downloading MNSIT dataset and assigning to dataset
                    transform=transforms.Compose([                          # Here we are composing different trasnformations
                        transforms.ToTensor(),                              # First transformation of Converting to Tensor
                        transforms.Normalize((0.1307,), (0.3081,))          # Second transformation of Normalizing data with mean and standard devication
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                          # Specifying batch size, Shuffle=True allows using different data every time
test_loader = torch.utils.data.DataLoader(                                  # Loading the test data. DataLoader is used to load the data which can be iterated.
    datasets.MNIST('../data', train=False, transform=transforms.Compose([   # Here we are composing different trasnformations
                        transforms.ToTensor(),                              # First transformation of Converting to Tensor
                        transforms.Normalize((0.1307,), (0.3081,))          # Second transformation of Normalizing data with mean and standard devication
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                          # Specifying batch size, Shuffle=True allows using different data every time


0it [00:00, ?it/s]

9920512it [00:01, 8883817.66it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134693.63it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:00, 2201617.41it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


8192it [00:00, 51980.98it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm                                                           # tqdm is a utility for displaying progress bar
def train(model, device, train_loader, optimizer, epoch):                       # Method that does actual training of a model using different things
    model.train()                                                               # Actual training of our neural network
    pbar = tqdm(train_loader)                                                   # Passing training data loader to the tqdm object for the progress bar
    for batch_idx, (data, target) in enumerate(pbar):                           # Enumerating the progress bar object that returns a tuple
        data, target = data.to(device), target.to(device)                       # Assigning data and target to device memory
        optimizer.zero_grad()                                                   # Zero_grad does clear(zero) all the gradients before back propagation in order to avoid adding to earlier gradients
        output = model(data)                                                    # We pass data to our model and get the output
        loss = F.nll_loss(output, target)                                       # nll_loss is a negative log loss function which is used to optimize the parameters of a network 
        loss.backward()                                                         # It computes gradient of loss for all parameters in loss that have requires_grad=True
        optimizer.step()                                                        # Its a Gradient descent
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  # Create a progress bar with loss and batch_id


def test(model, device, test_loader):                                                                   # Method that does actual training of a model using different things
    model.eval()                                                                                        # Evaluate a model with a score on how well it performed
    test_loss = 0                                                                                       # Test loss
    correct = 0                                                                                         # Accuracy
    with torch.no_grad():                                                                               # It will make all operations in the block have no gradients
        for data, target in test_loader:                                                                # Enumerate over test loader
            data, target = data.to(device), target.to(device)                                           # Assigning data and target to device memory
            output = model(data)                                                                        # We pass data to our model and get the output
            test_loss += F.nll_loss(output, target, reduction='sum').item()                             # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)                                                   # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()                                       # Compute Accuracy

    test_loss /= len(test_loader.dataset)                                                               # Compute Test loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(                       # Print Average loss and accuracy
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)                                                    # Create a neural network and assign it to device memory
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)            # Using SGD optimizer with learning rate of 0.01 and momentum of 0.9

for epoch in range(1, 2):                                                   # Enumerate over 1 epochs
    train(model, device, train_loader, optimizer, epoch)                    # Perform model training 
    test(model, device, test_loader)                                        # Perform model testing

loss=1.9513400793075562 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.71it/s]



Test set: Average loss: 1.8775, Accuracy: 2872/10000 (29%)

